### Kernighan-Lin Algorithm

The [Kernighan-Lin algorithm](https://en.wikipedia.org/wiki/Kernighan-Lin_algorithm) is a hill-climbing approximation to the _minimum balanced graph cut_ problem. Let's review what this means.
- _graph cut_: partition a graph into two classes
- _minimum_: such that the two classes minimize the number of edges between classes
- _balanced_: and such that the two classes have the same size, or approximately the same size. 

This tutorial discusses the parts of the algorithm using an implementation that was first created using code generated by Microsoft Copilot in response to the following prompt:

`can you show me python code for implementing Kernigahn and Lin's hill-climbing solution to finding balanced partitions with minimum edges cut`

The code was then modified in three ways:
- type hints were added
- type checks were added
- functions were refactored
- function descriptions were added
- variables were given more meaningful names

---

**Visualization**: The following code will be used for visualization

In [ ]:
#############
## Cell 1  ##
#############

import matplotlib.pyplot as plt
from matplotlib.axes import Axes
import networkx as nx
import numpy as np
import random
from typing import Hashable, Tuple, Set

def get_NCM_Figure3_14() -> Tuple[nx.Graph, dict[Hashable, Tuple[float, float]]]:
    """
        Figure 3.14 from the book Networks, Crowds, and Markets is a useful
        example graph. This function returns this figure as a networkx Graph
        and a position dictionary for the neato layout
    """
    G: nx.Graph = nx.Graph()
    G.add_nodes_from(range(0,14))
    G.add_edges_from([(0,1),(0,2),(1,2),(3,4),(3,5),(4,5),(8,9),(8,10),(9,10),(11,12),(11,13),(12,13),(2,6),(5,6),(7,8),(7,11),(6,7)])
    pos: dict[Hashable, Tuple[float, float]] = nx.nx_pydot.graphviz_layout(G,prog='neato')
    return G, pos

def draw_edge_by_type(G: nx.Graph, 
                      pos: dict[Hashable, Tuple[float, float]], 
                      edge: Tuple[Hashable, Hashable], 
                      partition: Tuple[Set, ...]
                      ) -> None:
    """
        Draw edges between nodes in different partitions using dashed lines.
        Draw edges between nodes within the same partition using solid lines.
    """
    edge_style = 'dashed'
    for part in partition:
        if edge[0] in part and edge[1] in part:
            edge_style = 'solid'
            break
    nx.draw_networkx_edges(G, pos, edgelist=[edge], style = edge_style)

def show_partitions(G: nx.Graph,
                    partition: Tuple[Set, ...], 
                    pos: dict[Hashable, Tuple[float, float]] | None = None,
                    title = ""
                    ) -> None:
    """ 
        Show the networkx graph with colors and edges indicating properties
        of the partition

        Edges:
        • Dashed lines indicate edges between nodes in different partitions
        • Solid lines indicate edges between nodes in the same partition

        Nodes:
        • All nodes in the same partition get mapped to the same color
        • When there are more partitions than ther are in the color pallette, repeat colors
    """
    #color_list = ['c','m','y','g','r']
    color_list: list[str] = ['y', 'lightblue', 'violet', 'salmon', 
                         'aquamarine', 'lightpink', 'lightgray', 'linen']
    plt.clf()
    ax: Axes = plt.gca()
    if pos is None: 
        pos = nx.spring_layout(G, seed = 0)
    for i in range(len(partition)):
        nx.draw_networkx_nodes(partition[i],pos,node_color=color_list[i%len(color_list)], alpha = 0.8)
    for edge in G.edges:
        draw_edge_by_type(G, pos, edge, partition)
    nx.draw_networkx_labels(G,pos)

    ax.set_title(title)
    ax.set_axis_off()

---

**Initialization**: Since we want a balanced graph cut, initialization should partition the graph vertices into two equal-sized groups (to within one, since there might be an odd number of vertices).

In [ ]:
#############
## Cell 2  ##
#############

def initialize_partition(G: nx.Graph,
                         seed: int | None = None
                         ) -> Tuple[set[Hashable], set[Hashable]]:
    """
        Input: networkx undirected graph
        Output: two sets with the graph nodes split in half
    """
    # Check types
    if type(G) is not nx.Graph:
        raise TypeError("Requires undirected graph")
    
    # Initialize partitions
    nodes: list[Hashable] = list(G.nodes())
    if seed is not None:
        random.seed(seed)
    random.shuffle(nodes)
    mid: int = len(nodes) // 2
    A: set[Hashable] = set(nodes[:mid])
    B: set[Hashable] = set(nodes[mid:])

    return (A,B)


Visualize the initial partition for Figure 3.14 from Networks, Crowds, and Markets

In [ ]:
#############
## Cell 3  ##
#############

G, pos = get_NCM_Figure3_14()
partition = initialize_partition(G)
show_partitions(G, 
                partition=partition, 
                title="Fig 3.14 Initial partition",
                pos=pos)


Choose the initial partition with a fixed seed so that we can refer to it in the next steps of the algorithm.

In [ ]:
#############
## Cell 4  ##
#############
seed: int = 1234
partition = initialize_partition(G, seed=seed)
show_partitions(G, 
                partition=partition, 
                title="Fig 3.14 Initial partition",
                pos=pos)

**Count the number of edges cut**: We'll need a function that counts the number of edges cut, so let's write that now. We'll then add that information to the graph title.

In [ ]:
#############
## Cell 5  ##
#############
def count_edges_cut(G: nx.Graph,
                    group_A: set[Hashable],
                    group_B: set[Hashable]
                    ) -> int:
    """ 
        Count the number of edges cut if the nodes in graph G are split into 
        group A and group B
    """
    cut_size:int = 0
    for u in group_A:
        for v in G.neighbors(u):
            if v in group_B:
                cut_size += 1
    return cut_size

Show initial partition with the number of edges cut in the title.

In [ ]:
#############
## Cell 6  ##
#############
show_partitions(G, 
                partition=partition, 
                title=f"Fig 3.14: Initial partition with {count_edges_cut(G, partition[0], partition[1])} edges cut",
                pos=pos)

---

**Cost for moving a vertex from one group to another**: We now need to figure out the cost for moving a node from one group in the partition to another. When we move vertex $a\in A$ to group $B$, two things happen:
- Edges from vertex $a$ in group $A$ to nodes $b$ in group $B$ that were cut are no longer cut since nodes $a$ and $b$ will both be in the same group. Denote the _gain_ (or improvement) in the number of edges cut by $E(a,A)$. "Gain" indicates that we are improving by decreasing the number of edges cut. The $E$ indicates edges between vertex $a$ and vertices that are "external" to vertex $a$ because they are not in $a$'s group (denoted by $A$) but rather in the other group (denoted by $B$).
- Edges from vertex $a$ to nodes $c\in A$ that were cut are now cut since nodes $a$ and $c$ will no longer be in the same group. Denote the _loss_ in edges cut by $I(a,A)$, where the $I$ indicates edges "internal" to vertex $a$'s original group, which is denoted by $A$.

The net gain for moving vertex $a$ from group $A$ to group $B$ is denoted by

$$D(a,A) = E(a,A) - I(a,A)$$

where the $D$ stands for "Delta". Thus, $D(a,A)$ counts the net change in edges cut if we move vertex $a$ from its group $A$ in the partition to the other group.

Let's write the code for computing $D(a,A)$.

In [ ]:
#############
## Cell 7  ##
#############
def gain(G: nx.Graph,
         u: Hashable, 
         group_A: set[Hashable], 
         group_B: set[Hashable]
         ) -> int:
    """ 
        count the net gain in the number of edges cut if we swap node u
        from group A to group B. See D_a from https://en.wikipedia.org/wiki/Kernighan-Lin_algorithm
    """

    internal = sum(1 for v in G.neighbors(u) if v in group_A)
    external = sum(1 for v in G.neighbors(u) if v in group_B)
    return external - internal

Since I chose the seed used to initialize the initial partition, I know that vertex 13 is in a different partition from its two neighbors. Thus, if we move vertex 13 from its current group to the other, we should see an net increase of 2.

In [ ]:
#############
## Cell 8  ##
#############
print(f"Moving vertex 13 removes {gain(G, 13, partition[0], partition[1])} edges from the graph cut")

The opposite happens if we move vertex 5 because that will put it in a different group than its three neighbors.

In [ ]:
#############
## Cell 9  ##
#############
print(f"Moving vertex 5 removes {gain(G, 5, partition[0], partition[1])} edges from the graph cut")


Importantly, we can also see what happens if we move a node from the other group in the partition. Let's choose vertex 7 because that will decrease the number of cut edges. Note that we'll need to swap the order of the partitions passed to the `gain` function since vertex 7 is in the second partition.

In [ ]:
print(f"Moving vertex 7 removes {gain(G, 7, partition[1], partition[0])} edge from the graph cut")


---

**Swapping Vertices**: Since we are doing a balanced cut, anytime we move one vertex from group $A$ to group $B$, another vertex has to be moved from group $B$ to group $A$. We'll say that vertex $a$ and vertex $b$ are _swapped_. In order to hill-climb, we need to know whether partition after the swap has fewer cut edges than the partition before the swap. 

There is a something a little tricky going on. This is a hill-climbing algorithm, so we want to swap vertices that improve the score. An improved score means that there are fewer edges cut. Thus, a positive gain means fewer edges cut.

The improvement in the score before the swap and after the swap is given by

$$T_{\rm old} - T_{\rm new} = D(a,A) + D(b,B) - 2\delta(\{a,b\}\in E)$$

The first two parts of this equation are pretty easy to understand. They say
- $D(a,A)$ represents the gain if we move $a$ from group $A$ to group $B$.
- $D(b,B)$ represents the gain if we move $b$ from group $B$ to group $A$.

The $\delta$ is more difficult to understand. Let's define it formally

$$
    \delta(\{a,b\}\in E) = 
        \begin{array}{ll}
            1 & {\rm if\ edge\ } \{a,b\} {\rm \ is\ in\ the\ edge\ set\ } E \\
            0 & {\rm otherwise}
        \end{array}
$$

Here's why the $\delta$ is needed. Vertex $a$ and vertex $b$ are in different groups in the partition. This means that if there is an edge between them, moving vertex $a$ from group $A$ to group $B$ means that the $a$ and $b$ are now both in group $B$ because the edge $\{a,b\}$ is no longer cut, so it contributes to the gain $D(a,A)$. Additionally, moving $b$ from group $B$ to group $A$ contributes to the gain $D(b,B)$. But after the swap, the two vertices are still not in the same groups edge $\{a,b\}$ is still cut. Because it's still cut, we don't want to count it as part of the gain. And since both $D(a,A)$ and $D(b,B)$ counted it, we have to subtract it twice. 

Let's write the code. Note that the code from Copilot didn't include $\delta$ so I had to fix that.




In [ ]:
def gain_from_swap(G: nx.Graph,
                   u: Hashable,
                   v: Hashable,
                   group_A: set[Hashable],
                   group_B: set[Hashable]
                   ) -> int:
    """ 
        Compute the net gain from swapping a and b using the equation
        T_{old} - T_{new} = D(a,A) + D(b,B) - 2 delta({a,b} in E)
    """
    gain_u: int = gain(G, u, group_A, group_B)
    gain_v: int = gain(G, v, group_B, group_A)
    delta: int = int(v in G.neighbors(u))

    return gain_u + gain_v - 2*delta

            

Swapping vertex 13 and vertex 7 should improve the score because that will remove two cut edges between vertex 13 and its neighbors, and also remove net one cut edge between vertex 7 and it's neighbors.

In [ ]:
print(f"Swapping 7 and 13 removes {gain_from_swap(G, 13, 7, partition[0], partition[1])} edges")

---

**Stopping Criterion**: This is a hill-climbing algorithm, so swapping vertices until the gain from the swap becomes zero of negative. This is encoded by setting the score of the best swap to 0 outside the optimization loop. 

Copilot also wrapped the code inside a four loop, presumably because it could run a really long time for large graphs.

Wrapping this all into an optimization loop gives the following code. 

In [ ]:
def kernighan_lin_bisection(G: nx.Graph, 
                            max_iter: int=10,
                            seed: int | None = None
                            ) -> Tuple[set[Hashable], set[Hashable], int]:
    """
        Input: undirected graph
    """
    # Check types
    if type(G) is not nx.Graph:
        raise TypeError("Requires undirected graph")
    
    # Initialize
    group_A, group_B = initialize_partition(G, seed)
    
    # Compute scores for all swaps
    for _ in range(max_iter):
        gains: list[Tuple, int, int] = []
        for u in group_A:
            for v in group_B:
                swap_score: int = gain_from_swap(G, u, v, group_A, group_B)
                gains.append((swap_score, u, v))
        
        gains.sort(reverse=True)
        
        best_gain: int = 0
        best_pair: tuple[Hashable, Hashable] | None = None
        current_gain: int = 0
        for gain_value, u, v in gains:
            current_gain += gain_value
            if current_gain > best_gain:
                best_gain = current_gain
                best_pair = (u, v)
        
        if best_pair is not None:
            u, v = best_pair
            group_A.remove(u)
            group_B.add(u)
            group_B.remove(v)
            group_A.add(v)
        else:
            break
    
    return group_A, group_B, count_edges_cut(G,group_A, group_B)

In [ ]:
# Example usage
G, pos = get_NCM_Figure3_14()
A, B, cut_size = kernighan_lin_bisection(G, max_iter=int(len(G.nodes)/2), seed=1234)
print("Partition A:", A)
print("Partition B:", B)
print("Cut size:", cut_size)
show_partitions(G, 
                partition=[A, B], 
                title=f"Fig 3.14: Final partition with {cut_size} edges cut",
                pos=pos)

That looks awesome! But there is a problem.

---

**Local Maxima**: Let's repeat but with a different seed

In [ ]:
# Example usage
G, pos = get_NCM_Figure3_14()
A, B, cut_size = kernighan_lin_bisection(G, max_iter=int(len(G.nodes)/2), seed=121)
print("Partition A:", A)
print("Partition B:", B)
print("Cut size:", cut_size)
show_partitions(G, 
                partition=[A, B], 
                title=f"Fig 3.14: Final partition with {cut_size} edges cut",
                pos=pos)

Here, we see the problem with hill-climbing: the initial partition determines how well the problem is solved. A standard approach for dealing with local maxima is to do several trials and choose the best answer. Let's apply this to the karate club graph. We'll first run it with a known seed, and then put it into a loop with several starting points.

In [ ]:
#############
## Cell 10 ##
#############
G = nx.karate_club_graph()
A, B, cut_size = kernighan_lin_bisection(G, max_iter=int(len(G.nodes)/2), seed=1234)
print("Partition A:", A)
print("Partition B:", B)
print("Cut size:", cut_size)
show_partitions(G, 
                partition=[A, B], 
                title=f"Karate Club Graph: Final partition with {cut_size} edges cut")

In [ ]:
#############
## Cell 10 ##
#############
num_trials: int = 10
best_A, best_B, best_cut_size = kernighan_lin_bisection(G, max_iter=int(len(G.nodes)/2))
for i in range(num_trials):
    A, B, cut_size = kernighan_lin_bisection(G, max_iter=int(len(G.nodes)/2))
    print(f"Cut size for iteration {i} = ", cut_size)
    if cut_size<best_cut_size:
        best_A = A
        best_B = B
        best_cut_size = cut_size
show_partitions(G, 
                partition=[best_A, best_B], 
                title=f"Karate Club Graph: Final partition with {best_cut_size} edges cut")

The smallest value I found doing this was 10 cut edges.

---

Student activities
- What is the smallest value you found for the cut size of the karate graph?
- What is the largest value you found for the cut size of the karate graph?
- What do you learn from this about the Kernighan-Lin algorithm?